# 3D Gif Generator

In [ ]:
import numpy as np

In [ ]:
# === Settings ===
input_path = "./viz/tsdf-rgbd-04.ply"           # <-- Path to the input .ply file to be visualized
output_dir = "spin_frames"                      # <-- Output Directory
gif_path = "spinning_mesh3.gif"                 # <-- Output Name
num_frames = 50                                # <-- Number of frames
rotation_axis = [0, 1, 0]                       # <-- Axis to be rotated in
degrees_per_frame = 360 / num_frames            # <-- Degrees per frame
img_size = (512, 512)                           # <-- Image size

In [ ]:
import open3d as o3d
import numpy as np
import imageio
import os
import copy

# === Load and prepare mesh ===
mesh = o3d.io.read_triangle_mesh(input_path)
mesh.compute_vertex_normals()

if mesh.has_vertex_colors():
    print("Mesh has vertex colors.")
    colors = np.asarray(mesh.vertex_colors)
    if colors.max() > 1.0:
        mesh.vertex_colors = o3d.utility.Vector3dVector(colors / 255.0)

# === Setup visualizer ===
vis = o3d.visualization.Visualizer()
vis.create_window(visible=False, width=img_size[0], height=img_size[1])
vis.add_geometry(mesh)

ctr = vis.get_view_control()
ctr.set_zoom(0.7)

os.makedirs(output_dir, exist_ok=True)
images = []

# === Render spin frames ===
current_mesh = mesh
for i in range(num_frames):
    mesh_copy = copy.deepcopy(current_mesh)

    angle = np.deg2rad(degrees_per_frame * i)
    R = mesh_copy.get_rotation_matrix_from_axis_angle(np.array(rotation_axis) * angle)
    mesh_copy.rotate(R, center=mesh_copy.get_center())

    vis.clear_geometries()
    vis.add_geometry(mesh_copy)

    vis.poll_events()
    vis.update_renderer()

    img_path = os.path.join(output_dir, f"frame_{i:03d}.png")
    vis.capture_screen_image(img_path)
    images.append(imageio.imread(img_path))

vis.destroy_window()

# === Create GIF ===
imageio.mimsave(gif_path, images, duration=0.05)
print(f"✅ GIF saved as: {gif_path}")

Mesh has vertex colors.


C:\Users\tayla\AppData\Local\Temp\ipykernel_6380\2540142956.py:54: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(img_path))


✅ GIF saved as: spinning_mesh.gif
